# A notebook to read a CSV template with some features and update it to elab

Please enter file name in the following cell, and what is the delimiter in that file

In [1]:
CSVfilename="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/UploadData/DirectResolution/RawTables/2022-03-22_Subsistence.tsv"
delimiterFile="\t"

Please enter the one-line file where your token is saved in the following cell

In [2]:
tokenFile="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/credentials/tokenELAB"

Now preparing all required python libs

In [3]:
import os
import json
import requests
import csv
import pandas
import numpy
from apiclient import discovery, errors
from httplib2 import Http
from oauth2client import client, file, tools
import os.path

token = format(open(tokenFile,"r").readline().strip())
url = "https://elab-dev.pasteur.fr/api/v1/"
headers1 = {'Authorization': token, 'Accept': 'application/json','Content-Type':'application/json'}
headers2 = {'Authorization': token, 'Accept': 'application/json'}
params={}





Reading the data. SkelDict is to make sure thta if eLab configuration change we just need to change here and not the excel template. 

In [6]:

DICT={"Site":{
        "Name":"Site",
        "Description":"None",
        "Note":"None",
        "Amount":"fixed_1",
        "Unit":"fixed_unit",
        "Main geographic region":"Site_Main Geographic Region",
        "Country":"Site_Country",
        "Province / Region":"Site_Province / Region",
        "Locality":"Site_Locality",
        "Latitude":"Site_LatChanged",
        "Longitude":"Site_LongChanged",
        "Site type":"Site_Site type",
        "Pictures":"None",
        "parentSampleID":"None"
    },

    "Individual":{
        "Name":"Individual",
        #"Description":"Individual_Archaeologist Observations",
        "Description":"None",
        "Note":"None",
        "Amount":"fixed_1",
        #"Unit":"fixed_Unit | pcs",
        "Unit":"fixed_unit",
        "parentSampleID":"Site",
        "Archaeologist ID":"Individual_Archaeologist ID",
        "Archaeologist group":"Individual_Archaeologist group",
        "Site Name":"Site",
        "Date":"Individual_Date",
        "Datation method":"Individual_Datation method",
        "Subsistence Strategy": "Individual_Subsistence Strategy",
        "Age":"Individual_Age",
        "Gender":"Individual_Gender",
        "Pictures":"None",
        "Linked individuals":"None"
    },

    "Skeleton Element":{"Name":"Skeleton Element",
         "From Individual":"Individual",
         "Description":"Skeleton Element_description",
         "Note":"None",
         "Amount":"fixed_1",
         #"Unit":"fixed_Unit | pcs",
         "Unit":"fixed_unit",
         "parentSampleID":"Individual",
         "Archaeologist sample ID":"Skeleton Element_Archaeologist sample ID",
         "Pictures Labelling":"Skeleton Element_Pictures Labelling",
         "Pictures Drilling":"None",
         "Bone type":"Skeleton Element_Bone type",
         "Skeleton element":"Skeleton Element_Skeleton element",
         "Exportation Permit Number":"Skeleton Element_Exportation Permit Number",
         "Observation Labelling":"Skeleton Element_Observation Labelling",
         "Observation Scanning":"Skeleton Element_Observation Scanning",
         "Observation Drilling":"None",
         "Scanned":"Skeleton Element_Scanned"

         
    }
}




Table=pandas.read_csv(CSVfilename,delimiter=delimiterFile)
ListTypes=[i.split('_', 1)[0] for i in Table]
ListTypes=list(dict.fromkeys(ListTypes))
print(ListTypes)


['Individual']


Prepare all the eLab-API keys necessary to down and upload data

In [7]:
def BadRequest(myReq,code=200):
    return(myReq.status_code !=code)


r = requests.get(url + "sampleTypes", headers = headers2)
if BadRequest(r,200):
    r.raise_for_status()
data = r.json()
types = {}
for typ in data.get("data"):
    types[format(typ.get("name"))] = format(typ.get("sampleTypeID"))

print(types)


FeateLab={}
for sampTY in ListTypes:
    r = requests.get(url + "sampleTypes/" + types[sampTY] + "/meta", headers = headers2)
    if BadRequest(r,200):
        r.raise_for_status()
    data = r.json()
    FeateLab[sampTY] = {}
    for feat in ['Name','Description','Note','Amount','Unit',"parentSampleID"]:
        FeateLab[sampTY][feat] = {"ID": "notMeta"}
    for feat in data.get("data"):
        FeateLab[sampTY][format(feat.get("key"))] = { "ID":format(feat.get("sampleTypeMetaID")),
                                              "TYPE":format(feat.get("sampleDataType"))}

print(FeateLab)

{'Individual': '39466', 'Site': '39468', 'Skeleton Element': '39469', 'Extract': '39470', 'Indexed Library': '39494', 'Library pool': '39495', 'Non Indexed Library': '39556', 'Bone pellet': '39599'}
{'Individual': {'Name': {'ID': 'notMeta'}, 'Description': {'ID': 'notMeta'}, 'Note': {'ID': 'notMeta'}, 'Amount': {'ID': 'notMeta'}, 'Unit': {'ID': 'notMeta'}, 'parentSampleID': {'ID': 'notMeta'}, 'Archaeologist group': {'ID': '244121', 'TYPE': 'CHECKBOX'}, 'Archaeologist ID': {'ID': '244132', 'TYPE': 'TEXT'}, 'Site Name': {'ID': '244133', 'TYPE': 'SAMPLELINK'}, 'Date': {'ID': '244134', 'TYPE': 'TEXT'}, 'Datation method': {'ID': '244135', 'TYPE': 'TEXTAREA'}, 'Age': {'ID': '244136', 'TYPE': 'TEXT'}, 'Gender': {'ID': '244137', 'TYPE': 'COMBO'}, 'Linked individuals': {'ID': '244138', 'TYPE': 'TEXT'}, 'Pictures': {'ID': '244141', 'TYPE': 'FILE'}, 'Subsistence Strategy': {'ID': '246288', 'TYPE': 'TEXT'}}}


Check the columns in  Table are recognized here. If no lines in output, you are just fine.

In [8]:
for sampTY in  FeateLab.keys():
    for feat in FeateLab[sampTY].keys():
        if feat not in DICT[sampTY].keys():
            exit(sampTY+": "+feat + "--> NOT IN DICTIONARY")
        
    for feat in DICT[sampTY].keys():
        if feat not in FeateLab[sampTY].keys():
            exit(sampTY+": "+feat + "--> NOT IN eLAB")
        



We get all the possible values for checkboxes and dropdown features of Extracts and check our extractTable table is fine. If no lines in output, you're just fine


In [9]:
for sampTY in FeateLab.keys():
    r = requests.get(url + "sampleTypes/" + types[sampTY] + "/meta", headers = headers2)
    data = r.json()
    for feat in data.get("data"):
        key=feat.get("key")
        if key not in DICT[sampTY].values():
            continue
        if feat.get("sampleDataType") == "CHECKBOX" or feat.get("sampleDataType") == "COMBO":
            OptionELAB=feat.get("optionValues")
            if DICT[sampTY][key].startswith("fixed"):
                tabVal=DICT[sampTY][key].split("_")[1]
                if tabVal not in OptionELAB:
                    exit(sampTY+": " + tabVal + "-- not mapped in eLab for " + key)
            else:
                Table.loc[Table[DICT[sampTY][key]].isnull(),DICT[sampTY][key]]="NA"
                for tabVal in Table[DICT[sampTY][key]].unique():
                    if tabVal not in OptionELAB:
                        exit(sampTY+": " + tabVal + "-- not mapped in eLab for " + key)




Now, we make the json for each extract and we upload or update in eLab!
Change the default prompt line:
- put y if you are sure you want to overwrite already loaded info in eLab, 
- put n if you are sure you want to leave already loaded info in eLab (although it doesn't match info in your table)
- put anything else if you want a case by case prompt

In [10]:
####get all registered skeleton element and extracts
registered = {}
for name in ListTypes:
    #print(name)
    r = requests.get(url + "samples" , headers = headers2, params = {'sampleTypeID': types[name]})
    if BadRequest(r,200):
        r.raise_for_status()
    data = r.json()
    myList = {}
    for sam in data.get("data"):
        if format(sam.get("name")) in myList.keys():
            print(name + ": " + sam.get("name") + " duplicated")
            break
        myList[format(sam.get("name"))]=format(sam.get("sampleID"))
    registered[name] = myList




In [11]:
defaultPrompt="y"

#for sampTy in ["Site","Individual","Skeleton Element"]:
for sampTy in ListTypes:
    
    ###make an unique TableTYPE
    TableTYPE={}
    for fea in DICT[sampTy].values():
        if fea != "None" and not fea.startswith("fixed") and fea in Table.keys():
            TableTYPE[fea]=Table[fea]
        
    TableTYPE=pandas.DataFrame(TableTYPE).drop_duplicates()
    
    ###iterate over extracts in table
    for index,name in TableTYPE[DICT[sampTy]['Name']].items():
        print(name)
        id=registered[sampTy][name]
        r=requests.get(url + "samples/"+id, headers = headers2)
        if BadRequest(r,200):
            r.raise_for_status()
        r=r.json()
        ###change to lower case all the keys because API sometimes use upper, lower for different request (A MESS!)
        dataLoaded={}
        for oldkey in r:
            newkey=oldkey.lower()
            dataLoaded[newkey] = r[oldkey]


        ###get loaded metadata
        MDR=requests.get(url + "samples/"+id+"/meta", headers = headers2)
        if BadRequest(MDR,200):
            MDR.raise_for_status()
        data=MDR.json().get("data")
        metaLoaded={}
        for i in data:
            metaLoaded[i["key"]]=str(i["value"])
        
        ####prepare the Data to be loaded
        Data={}
        for fea in FeateLab[sampTy].keys():
            element="???"
            if DICT[sampTy][fea] not in TableTYPE.keys():
                element="None"
            else:
                ###upload Data 
                if fea not in ["Name","Amount","Unit"]:
                    if FeateLab[sampTy][fea]['ID'] == "notMeta":
                        ###fixed value (from dico)
                        if DICT[sampTy][fea].startswith("fixed"):
                            element=DICT[sampTy][fea].split("_")[1]
                        ###WHEN NO VALUE to enter
                        elif DICT[sampTy][fea]=="None":
                            element="None"
                        elif fea == "parentSampleID":
                            if sampTy == "Site":
                                element="None"
                            else:
                                element=registered[DICT[sampTy]["parentSampleID"]][TableTYPE[DICT[sampTy]["parentSampleID"]][index]]
                        else:
                            element=TableTYPE[DICT[sampTy][fea]][index]

                        if element != "None":
                            ###check delta when patching
                            elementLoaded=dataLoaded[fea.lower()]
                            if format(elementLoaded) != format(element):
                                print("For "+sampTy+" "+name+", do you want to update the "+fea+" field? That is: "+format(element)+ " vs what already loaded: "+format(elementLoaded))
                            if DICT[sampTy][fea]=="None":
                                print("no update for None features")
                                prompt="n"
                            else:
                                prompt=defaultPrompt
                                while prompt not in ["y","n"]:
                                    prompt = input("replace y/n??")
                                if prompt == "n":
                                    element="None"
                    
                            if element != "None":
                                Data[fea]=element
            
                    elif FeateLab[sampTy][fea]['ID'] != "notMeta" and FeateLab[sampTy][fea]['TYPE'] != "FILE":
                        ###fixed value (from dico)
                        if DICT[sampTy][fea].startswith("fixed"):
                            element=DICT[sampTy][fea].split("_")[1]
                            MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
                        elif DICT[sampTy][fea]=="None":
                            element="NA"
                            MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
                        elif (fea == "From Individual" and sampTy =="Skeleton Element") or (fea == "Site Name" and sampTy =="Individual"):
                            sisi=TableTYPE[DICT[sampTy][fea]][index]
                            IDsisi=registered[DICT[sampTy][fea]][sisi]
                            element=sisi+"|"+IDsisi
                            samples={"sampleID": IDsisi,"name": sisi}
                            MetaData={
                                "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                                "sampleDataType": FeateLab[sampTy][fea]['TYPE'],
                                "samples": samples,
                                "key": fea,
                                "value": element
                            }
                        elif DICT[sampTy][fea] in TableTYPE.keys():
                            element=TableTYPE[DICT[sampTy][fea]][index]
                        if format(element)=="nan" or format(element)=="" or format(element)==" ":
                                element="NA"
                        MetaData={"key": fea,
                            "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                            "value": element,
                            "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
            
                        ###check if this is a new entry or not
                        ###check if new element is similar to what already loaded
                        needToPatch=False
                        if fea not in metaLoaded.keys(): 
                            needToPatch=True
                        elif metaLoaded[fea] != str(element):
                            print("difference for " + name + "(feature: " + fea + ") " + str(element) + " vs loaded : " + metaLoaded[fea])
                            if DICT[sampTy][fea]=="None":
                                print("no update for None features")
                                prompt="n"
                            else:
                                prompt=defaultPrompt
                                while prompt not in ["y","n"]:
                                    prompt = input("???replace y/n??")
                            if prompt == "y":
                                needToPatch=True
                        else:
                            needToPatch=True
                        if needToPatch:
                            #print(MetaData)      
                            MDR=requests.put(url + "samples/"+id+"/meta", headers = headers2,data = MetaData)
                            ####check the MetaData loading was correct
                            if BadRequest(MDR,204):
                                MDR.raise_for_status()
                        
        
                    if len(Data) == 0:
                        print("no data to be uploaded")
                    else:
                        DR=requests.patch(url + "samples/"+id, headers = headers2,data = Data)
                        if BadRequest(DR,204):
                            DR.raise_for_status()
print("finished")

AR0036
no data to be uploaded
AR0037
difference for AR0037(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0038
difference for AR0038(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0079
difference for AR0079(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0081
difference for AR0081(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0083
difference for AR0083(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0085
difference for AR0085(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0087
difference for AR0087(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0089
difference for AR0089(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0091
difference for AR0091(feature: Subsistence Strategy) NA vs loaded : nan
no data to be uploaded
AR0093
difference for AR0093(feature: Subsistence St

difference for AR0171(feature: Subsistence Strategy) agriculturalists? vs loaded : 
no data to be uploaded
AR0172
difference for AR0172(feature: Subsistence Strategy) agriculturalists? vs loaded : 
no data to be uploaded
AR0173
difference for AR0173(feature: Subsistence Strategy) agriculturalists? vs loaded : 
no data to be uploaded
AR0174
difference for AR0174(feature: Subsistence Strategy) agriculturalists? vs loaded : 
no data to be uploaded
AR0175
difference for AR0175(feature: Subsistence Strategy) agriculturalists? vs loaded : 
no data to be uploaded
AR0176
difference for AR0176(feature: Subsistence Strategy) agriculturalists? vs loaded : 
no data to be uploaded
AR0177
difference for AR0177(feature: Subsistence Strategy) agriculturalists? vs loaded : 
no data to be uploaded
AR0178
difference for AR0178(feature: Subsistence Strategy) hunter-gatherers vs loaded : 
no data to be uploaded
AR0179
difference for AR0179(feature: Subsistence Strategy) hunter-gatherers vs loaded : 
no dat

no data to be uploaded
AR0248
difference for AR0248(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0249
difference for AR0249(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0250
difference for AR0250(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0251
difference for AR0251(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0252
difference for AR0252(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0253
difference for AR0253(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0254
difference for AR0254(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0255
difference for AR0255(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0256
difference for AR0256(feature: Subsistence Strategy) CR/M vs loaded : 
no data to be uploaded
AR0257
difference for AR0257(feature: Subsistence Strategy) CR/M vs 

difference for AR0332(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0333
difference for AR0333(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0334
difference for AR0334(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0335
difference for AR0335(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0336
difference for AR0336(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0337
difference for AR0337(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0338
difference for AR0338(feature: Subsistence Strategy) PC vs loaded : 
no data to be uploaded
AR0339
difference for AR0339(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0340
difference for AR0340(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0341
difference for AR0341(feature: Subsistence Strategy) CR vs loaded : 
no data to be uploaded
AR0342
difference

no data to be uploaded
AR0560
no data to be uploaded
AR0561
no data to be uploaded
AR0562
no data to be uploaded
AR0563
no data to be uploaded
AR0564
no data to be uploaded
AR0565
no data to be uploaded
AR0566
no data to be uploaded
AR0567
no data to be uploaded
AR0568
no data to be uploaded
AR0569
no data to be uploaded
AR0570
no data to be uploaded
AR0571
no data to be uploaded
AR0572
no data to be uploaded
AR0573
no data to be uploaded
AR0574
no data to be uploaded
AR0575
no data to be uploaded
AR0576
no data to be uploaded
AR0577
no data to be uploaded
AR0578
no data to be uploaded
AR0579
no data to be uploaded
AR0580
no data to be uploaded
AR0581
no data to be uploaded
AR0582
no data to be uploaded
AR0583
no data to be uploaded
AR0584
no data to be uploaded
AR0585
no data to be uploaded
AR0586
no data to be uploaded
AR0587
no data to be uploaded
AR0588
no data to be uploaded
AR0589
no data to be uploaded
AR0590
no data to be uploaded
AR0591
no data to be uploaded
AR0592
no data to